In [5]:
# set autoreload
%load_ext autoreload
%autoreload 2
from models import MNISTConvNet, MNISTConvNetTrainer, get_mnist_dataloader
from torch.utils.data import DataLoader, Subset

train, val, test = get_mnist_dataloader(128)
# sample 1000 images from the training set
train = DataLoader(Subset(train.dataset, range(1000)))

# print some sample images from the training set with matplotlib
# import matplotlib.pyplot as plt
# fig, axs = plt.subplots(2, 5, figsize=(12, 3))
# for i, ax in enumerate(axs.flat):
#     ax.imshow(train.dataset[i][0].squeeze(), cmap='gray')
#     ax.set_title(f'Label: {train.dataset[i][1]}')

model   = MNISTConvNet()
trainer = MNISTConvNetTrainer(model, train, val, device='cpu')

trainer.train(num_epochs=3)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Model created and moved to cpu


 33%|███▎      | 1/3 [00:01<00:02,  1.13s/it]

Epoch 1, Loss: 1.0614983425301034, Accuracy: 0.635, Val Loss: 0.5571795701980591, Val Acc: 0.8117


 67%|██████▋   | 2/3 [00:02<00:01,  1.12s/it]

Epoch 2, Loss: 0.40281935161715593, Accuracy: 0.862, Val Loss: 0.366889625787735, Val Acc: 0.8834


100%|██████████| 3/3 [00:03<00:00,  1.11s/it]

Epoch 3, Loss: 0.27232630198871904, Accuracy: 0.904, Val Loss: 0.32728254795074463, Val Acc: 0.896


In [3]:
train

In [6]:
from models import MNISTFederatedServer

server = MNISTFederatedServer(2, 100, 0.001, model, train, val, test)
server.start_train()

Training client 0
Local Epoch 1, Loss: 0.1268272379057857, Accuracy: 0.961, client: 0, total: 1000
Local Epoch 2, Loss: 0.10452932583852291, Accuracy: 0.964, client: 0, total: 1000
Local Epoch 3, Loss: 0.09242231046153375, Accuracy: 0.97, client: 0, total: 1000
Local Epoch 4, Loss: 0.08285702297370963, Accuracy: 0.976, client: 0, total: 1000
Local Epoch 5, Loss: 0.07500874344255715, Accuracy: 0.975, client: 0, total: 1000
Training client 1
Local Epoch 1, Loss: 0.1268272379057857, Accuracy: 0.961, client: 1, total: 1000
Local Epoch 2, Loss: 0.10452932583852291, Accuracy: 0.964, client: 1, total: 1000
Local Epoch 3, Loss: 0.09242231046153375, Accuracy: 0.97, client: 1, total: 1000
Local Epoch 4, Loss: 0.08285702297370963, Accuracy: 0.976, client: 1, total: 1000
Local Epoch 5, Loss: 0.07500874344255715, Accuracy: 0.975, client: 1, total: 1000
Aggregating 2 local states
Round 0, Test Loss: 0.28992077708244324, Test Acc: 0.9197
Training client 0
Local Epoch 1, Loss: 0.06762130718327689, Acc

KeyboardInterrupt: 